In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from zipfile import ZipFile
file_name ="/content/gdrive/MyDrive/data/archive.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    image_path = os.path.join(os.getcwd(), '/content/Val-20200226T103730Z-001/Val/Annotations')
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('/content/Val-20200226T103730Z-001/Val/val.csv', index=None)
    print('Successfully converted xml to csv.')


main()

Successfully converted xml to csv.


In [ ]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    image_path = os.path.join(os.getcwd(), '/content/Train-20200226T103300Z-001/Train/Annotations')
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('/content/Train-20200226T103300Z-001/Train/train.csv', index=None)
    print('Successfully converted xml to csv.')


main()

Successfully converted xml to csv.


In [ ]:
!pip install pycocotools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3407, done.
remote: Counting objects: 100% (3407/3407), done.
remote: Compressing objects: 100% (2835/2835), done.
remote: Total 3407 (delta 891), reused 1389 (delta 515), pack-reused 0
Receiving objects: 100% (3407/3407), 34.97 MiB | 21.60 MiB/s, done.
Resolving deltas: 100% (891/891), done.


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd models/research
pip help install 


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

Description:
  Install packages from:
  
  - PyPI (and other indexes) using requirement specifiers.
  - VCS project urls.
  - Local project directories.
  - Local or remote source archives.
  
  pip also supports installing from "requirements files", which provide
  an easy way to specify a whole environment to be installed.

Install Options:
  -r, --requirement <file>    Install from the given requirements file. This
                              option can be used multiple times.
  -c, --constraint <file>     Constrain versions using the given constraints
                              file. This option can be used multiple times.
  --no-deps                   Don't inst

In [ ]:
pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 16.2 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 48 kB 5.5 MB/s 
     |████████████████████████████████| 4.1 MB 62.5 MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844512 sha256=7acfc922db5be376dff9b9ffca95078fbfc7d2527566d543eb2ae8e6ae4d41a2
  Stored in directory: /root/.cache/pip/wheels/71/7e/a2/461ab817fbaef68ec9cc60df16d3669d1285f032e4c98179bf
Successfully built tensorflow-object-detection-api


In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [ ]:
!wget https://raw.githubusercontent.com/NSTiwari/TensorFlow.js-Custom-Object-Detection/master/generate_tf_records.py -P /content/

--2022-07-11 22:13:25--  https://raw.githubusercontent.com/NSTiwari/TensorFlow.js-Custom-Object-Detection/master/generate_tf_records.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5136 (5.0K) [text/plain]
Saving to: ‘/content/generate_tf_records.py’

generate_tf_records 100%[===================>]   5.02K  --.-KB/s    in 0s      

2022-07-11 22:13:25 (56.2 MB/s) - ‘/content/generate_tf_records.py’ saved [5136/5136]



In [ ]:
!python generate_tf_records.py -l /content/gdrive/MyDrive/labelmap.pbtxt.txt -o /content/Train-20200226T103300Z-001/Train/train.record -i /content/Train-20200226T103300Z-001/Train/JPEGImages -csv /content/Train-20200226T103300Z-001/Train/train.csv
!python generate_tf_records.py -l /content/gdrive/MyDrive/labelmap.pbtxt.txt -o /content/Val-20200226T103730Z-001/Val/val.record -i /content/Val-20200226T103730Z-001/Val/JPEGImages -csv /content/Val-20200226T103730Z-001/Val/val.csv

INFO:Successfully created the TFRecords: /content/Train-20200226T103300Z-001/Train/train.record
INFO:Successfully created the TFRecords: /content/Val-20200226T103730Z-001/Val/val.record
